In [35]:
import pandas as pd
import numpy as np

In [96]:
uf = pd.read_csv("../data/train_data_uf.csv")

In [97]:
dc = pd.read_csv("../data/train_data_dc.csv")

In [98]:
uf.head()

,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,PhotoTop,PhotoDown,PhotoType,LayerTop,LayerDown,Rock,Carbonate,Ruin,Saturation,class,weight
0,Unload2,1001901,Field1,Well3,2621.0,2638.0,2624.0,2641.0,16.85,2.93,3.91,УФ,2.63,5.34,песчаник,с карбонатными обломками или конкрециями,частично разрушен,нефтенасыщенные,1,2.766972
1,Unload1,1001161,Field3,Well12,3175.1,3181.1,3176.3,3182.3,5.98,1.00,1.45,УФ,0.90,1.45,"пересл. песчаника, алевролита и глин",с примесью,не разрушен,не опред.,0,4.157229
2,Unload1,1001305,Field3,Well12,3252.0,3263.0,3253.5,3264.5,10.92,0.00,1.00,УФ,0.00,4.04,алевролит,слабокарбонатный,не разрушен,не опред.,0,8.989568
3,Unload1,1001475,Field3,Well9,798.0,805.0,798.7,805.7,6.90,6.72,6.90,УФ,6.72,6.90,алевро-аргиллит,не карбонатный,не разрушен,не опред.,0,8.772798
4,Unload2,1001097,Field8,Well14,2781.0,2799.0,2782.1,2800.1,17.99,15.98,16.99,УФ,12.00,17.81,кремнисто-глинистая порода,не карбонатный,не разрушен,битуминозный,2,2.436187


In [65]:
classes = pd.read_csv("../data/train_classes.csv")

In [66]:
ls


6_channel_transfer.ipynb      rn18-1e-3.ipynb      train-Copy2.ipynb
6_channel_transfer_SVM.ipynb  rn18-1e-5.ipynb      train.ipynb
EDA.ipynb                     rn18-ft.ipynb        train_data_dc.csv
autoencoder-big-1.ipynb       set_weights.ipynb    train_data_uf.csv
autoencoder-big-2.ipynb       to_multiclass.ipynb  transfer_baseline.ipynb
autoencoder.ipynb             to_train_test.ipynb  wandb/
hyper_opt.ipynb               train-Copy1.ipynb


In [67]:
classes.head()

,алевролит,аргиллит,глина,другое,карбонатная порода,кремнистая порода,переслой,песчаник
0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,1,1
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0


In [9]:
weights = dc.Rock.value_counts()

In [10]:
weights

песчаник                                          790
аргиллит                                          418
алевролит                                         378
пересл. песчаника, алевролита и глин              217
переслаивание песчаника и алевролита               48
кремнистая порода                                  47
песчаник глинистый                                 32
глинисто-кремнистая порода                         28
глина аргиллитоподобная                            21
карбонатно-кремнистая порода                       17
кремнисто-глинистая порода                         17
уголь                                              16
пересл. аргиллита и алевролита                     15
фундамент                                           9
алевро-аргиллит                                     8
алевролит сильноглинистый                           5
известняк                                           5
кремнисто-карбонатная порода                        4
пересл. песчаника и глины ар

In [11]:
classes.head()

,алевролит,аргиллит,глина,другое,карбонатная порода,кремнистая порода,переслой,песчаник
0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,1,1
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0


In [68]:
weights = classes.sum()

In [69]:
weights

алевролит              671
аргиллит               470
глина                  326
другое                  33
карбонатная порода      22
кремнистая порода      116
переслой               288
песчаник              1090
dtype: int64

In [158]:
def set_weights_to_file(path="../data"):
    """Sets weights in csv-files with uf-data, dc-data, classes (OHE). 
    
    Weights are applied on "weight" column.

    Args: 
        path: path to data with no slash. Example: '../data '
    """
    
    uf = pd.read_csv(f"{path}/train_data_uf.csv")
    dc = pd.read_csv(f"{path}/train_data_dc.csv")
    classes = pd.read_csv(f"{path}/train_classes.csv")
    uf, dc, _ = set_weights(uf, dc, classes)
    dc.to_csv(f"{path}/train_data_dc.csv", index=False)
    uf.to_csv(f"{path}/train_data_uf.csv", index=False)



def set_weights(uf, dc, classes):
    """Sets weights to kern classes.
    
    Computes weights of single classes as 1/(count_of_examples*const) 
    and after that sets final weights as mean of class weights to each example. 
    Weights are stored in "weight" column.
    
    Args:
        uf: pd.DataFrame with UF-data
        dc: pd.DataFrame with DL-data
        classes: pd.DataFrame with OHE-classes
        
    Returns:
        (uf,dc, weights_dict): dataframes with added(changed) "weight" column 
        and dict with weight values.
    """
    
    
    
    weights = classes.sum()
    weights_reduced = {}
    weights_list = {}

    for  i,j in weights.items():
        if i in ['алевролит', 'аргиллит', 'песчаник']:
            weights_reduced[i] = j*0.6
        if i=='аргиллит':
            weights_reduced[i] = j*0.71
        if i=='песчаник':
            weights_reduced[i] = j*0.78

    for i, j in (classes.sum().items()):
        #if i=='карбонатная порода' or i=='другое':
        #    weights_list[i]=1
        #else:     
        if i in weights_reduced.keys():
            weights_list[i]=classes.sum().sum()/weights_reduced[i]
        else:
            weights_list[i]=classes.sum().sum()/(abs(classes.sum().sum()-pd.Series(weights_reduced).sum()))
    def apply_weights(data, weights_dict=weights_list):
        """Sets weight as mean of given weights.
        Args: 
            data: row in dataframe
            weights_dict: dict with weight values

        Returns:
            float: computed weight
        """
        weight = 0
        weights = []
        for i,j in data.items():
            #print(i, weights_dict[i])  
            if(j!=0):
                weights.append(weights_dict[i])
        weight = (np.mean(weights))
        return weight
    
    uf["weight"]= classes.apply(apply_weights, axis=1)
    dc["weight"] = classes.apply(apply_weights, axis=1)
    return (uf, dc, weights_list)


In [159]:
set_weights_to_file()

In [121]:
weights_reduced

{'алевролит': 402.59999999999997, 'аргиллит': 329.0, 'песчаник': 850.2}

In [122]:
weights_list = {}
for i, j in (classes.sum().items()):
    #if i=='карбонатная порода' or i=='другое':
    #    weights_list[i]=1
    #else:     
    if i in weights_reduced.keys():
        weights_list[i]=classes.sum().sum()/weights_reduced[i]
    else:
        weights_list[i]=classes.sum().sum()/(abs(classes.sum().sum()-pd.Series(weights_reduced).sum()))

In [123]:
weights_list

{'алевролит': 7.491306507699951,
 'аргиллит': 9.167173252279635,
 'глина': 2.102914516803793,
 'другое': 2.102914516803793,
 'карбонатная порода': 2.102914516803793,
 'кремнистая порода': 2.102914516803793,
 'переслой': 2.102914516803793,
 'песчаник': 3.547400611620795}

In [125]:
np.array(list(weights_list.values())).mean()


3.840056619452418

In [126]:
weights_list

{'алевролит': 7.491306507699951,
 'аргиллит': 9.167173252279635,
 'глина': 2.102914516803793,
 'другое': 2.102914516803793,
 'карбонатная порода': 2.102914516803793,
 'кремнистая порода': 2.102914516803793,
 'переслой': 2.102914516803793,
 'песчаник': 3.547400611620795}

In [127]:
def apply_weights(data, weights_dict=weights_list):
    weight = 0
    weights = []
    for i,j in data.items():
        #print(i, weights_dict[i])  
        if(j!=0):
            weights.append(weights_dict[i])
    weight = (np.mean(weights))
    return weight

In [128]:
print(apply_weights(classes.iloc[1,], weights_list))

3.811134038232083


In [129]:
classes.apply(apply_weights, axis=1)

0       3.547401
1       3.811134
2       7.491307
3       8.329240
4       2.102915
          ...   
2084    3.547401
2085    3.811134
2086    3.811134
2087    3.547401
2088    3.547401
Length: 2089, dtype: float64

In [130]:
uf["weight"]= classes.apply(apply_weights, axis=1)
dc["weight"] = classes.apply(apply_weights, axis=1)

In [133]:
dc.to_csv('../data/train_data_dc.csv', index=False)
uf.to_csv('../data/train_data_uf.csv', index=False)